In [42]:
import numpy as np
import pandas as pd
import pprint
import google.generativeai as palm
import tiktoken
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [18]:
#Llama embedding model
from langchain.embeddings import LlamaCppEmbeddings
llama = LlamaCppEmbeddings(model_path=r'/Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf', n_batch=8, n_ctx=2048)
# llama = LlamaCppEmbeddings(model_path=r'/Users/kpathak/Downloads/llama-2-7b-chat.ggmlv3.q4_0.bin')
EMBEDDING_MODEL=llama

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 37632,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q4_0 

In [43]:
#Llama completion model
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path='/Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf',
    temperature=0,
    n_ctx=2048,
    max_tokens=4095,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)
COMPLETIONS_MODEL=llm

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 37632,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q4_0 

In [2]:
# text = "This is a test document."
# query_result = llama.embed_query(text)
# query_result


llama_print_timings:        load time =  3722.79 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =  3721.96 ms /     7 tokens (  531.71 ms per token,     1.88 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  3724.46 ms


[0.8761463165283203,
 -0.8497534394264221,
 1.0186741352081299,
 0.15613168478012085,
 0.09142674505710602,
 -0.412421852350235,
 0.014965352602303028,
 1.2169687747955322,
 0.09830321371555328,
 0.05218314379453659,
 0.6475121974945068,
 -0.24372020363807678,
 -0.13481348752975464,
 -0.5532916784286499,
 -0.6081416606903076,
 -0.14626061916351318,
 1.0144367218017578,
 -0.45640653371810913,
 -0.723211944103241,
 -0.157279834151268,
 -2.0107595920562744,
 0.11700478941202164,
 0.47858530282974243,
 -0.0386771596968174,
 0.8885231018066406,
 0.7793574929237366,
 -0.21672357618808746,
 1.0872336626052856,
 0.32295405864715576,
 0.3659127354621887,
 -0.972658097743988,
 0.303290456533432,
 0.4474232494831085,
 -0.3134004771709442,
 0.22290216386318207,
 0.305349737405777,
 0.4460129737854004,
 -0.33274614810943604,
 -5.122390270233154,
 0.16774901747703552,
 -0.2580338418483734,
 -0.2615836262702942,
 0.8798328638076782,
 0.09928571432828903,
 1.0043267011642456,
 1.509386658668518,
 -1.9

In [12]:
# len(query_result)

5120

### Reading open source data - from openai

In [7]:
#Reading open source data - from openai
df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df['content']=df['title']+' '+df['heading']+ ' '+df['content']
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
Athletics at the 2020 Summer Olympics – Women's shot put,Summary,Athletics at the 2020 Summer Olympics – Women'...,77
Switzerland at the 2020 Summer Olympics,Triathlon,Switzerland at the 2020 Summer Olympics Triath...,41
Rugby sevens at the 2020 Summer Olympics – Women's qualification,Africa,Rugby sevens at the 2020 Summer Olympics – Wom...,54
Ice hockey at the 2020 Winter Youth Olympics,Summary,Ice hockey at the 2020 Winter Youth Olympics S...,144
China at the 2020 Summer Olympics,Archery,China at the 2020 Summer Olympics Archery Chin...,88


### Getting embeddings

In [22]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = EMBEDDING_MODEL.embed_query(text)
    return result

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [23]:
#Takes longer time for running the embeddings
document_embeddings = compute_doc_embeddings(df.head(100))


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 396622.58 ms /   883 tokens (  449.18 ms per token,     2.23 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 396880.48 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 73150.60 ms /   163 tokens (  448.78 ms per token,     2.23 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 73196.47 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings: 


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 76423.09 ms /   173 tokens (  441.75 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 76470.10 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 112408.70 ms /   254 tokens (  442.55 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 112477.53 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings: 


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 60124.09 ms /   136 tokens (  442.09 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 60159.96 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 80910.23 ms /   183 tokens (  442.13 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 80960.44 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:   


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 125714.75 ms /   284 tokens (  442.66 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 125792.30 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 50829.16 ms /   115 tokens (  441.99 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 50860.91 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings: 


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 35446.55 ms /    80 tokens (  443.08 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =   455.37 ms /     1 runs   (  455.37 ms per token,     2.20 tokens per second)
llama_print_timings:       total time = 35925.60 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 96749.62 ms /   216 tokens (  447.91 ms per token,     2.23 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 96809.57 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:   


llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 126802.53 ms /   283 tokens (  448.07 ms per token,     2.23 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 126882.55 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 39962.05 ms /    88 tokens (  454.11 ms per token,     2.20 tokens per second)
llama_print_timings:        eval time =   487.50 ms /     1 runs   (  487.50 ms per token,     2.05 tokens per second)
llama_print_timings:       total time = 40477.72 ms

llama_print_timings:        load time =  4370.49 ms
llama_print_timings: 

### We could directly use the dataframe for QA, but because of the token limitations for the LLMs, we have to first get a most relevant context based on the question from the given dataframe,and then pass that with question in the formatted prompt to get the desired answer.

In [24]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [25]:
#Configs
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))


### Formatted prompt creation

In [26]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

### Getting desired answer 

In [46]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response=COMPLETIONS_MODEL(prompt=prompt)

    return response

In [47]:
response=answer_query_with_context("Why was the 2020 Summer Olympics originally postponed?", df, document_embeddings)
response

Selected 7 document sections:
("Boxing at the 2020 Summer Olympics – Women's featherweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Men's flyweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Men's featherweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Women's flyweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Men's heavyweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Men's lightweight", 'Schedule')
("Boxing at the 2020 Summer Olympics – Women's lightweight", 'Schedule')



llama_print_timings:        load time =  4370.49 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =  7697.68 ms /    16 tokens (  481.10 ms per token,     2.08 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  7706.18 ms
Llama.generate: prefix-match hit


 The 2020 Summer Olympics were originally postponed due to the COVID-19 pandemic.



llama_print_timings:        load time =  4297.09 ms
llama_print_timings:      sample time =    24.59 ms /    25 runs   (    0.98 ms per token,  1016.47 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 13038.09 ms /    25 runs   (  521.52 ms per token,     1.92 tokens per second)
llama_print_timings:       total time = 13141.78 ms


' The 2020 Summer Olympics were originally postponed due to the COVID-19 pandemic.\n'